In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [0]:
postal=pd.read_csv('Toronto_postalcode_canada.csv')

In [3]:
postal.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [0]:
geo_csv=pd.read_csv('Geospatial_Coordinates.csv')

In [6]:
geo_csv.rename(columns={'Postal Code': 'Postcode'},inplace=True)
geo_csv.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df_neigh=pd.merge(postal,geo_csv,on='Postcode')
df_neigh.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [0]:
#EXPORT TO .CSV
df_neigh.to_csv('Toronto_Postalcode_geo.csv')

In [0]:
import folium
from sklearn.cluster import KMeans

In [10]:
df=pd.read_csv('Toronto_Postalcode_geo.csv',index_col=0)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
#print total number of borough and neighborhood
print('DataFrame consists of {} broughs and {} neighbour '.format(len(df['Borough'].unique()),df.shape[0]))

DataFrame consists of 11 broughs and 103 neighbour 


In [12]:
#counting Bourough and Neighbour
df.groupby('Borough').count()['Neighbourhood']

Borough
Central Toronto      9
Downtown Toronto    18
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Queen's Park         1
Scarborough         17
West Toronto         6
York                 5
Name: Neighbourhood, dtype: int64

In [13]:
df_tor=df[df['Borough'].str.contains('Toronto')]
df_tor.reset_index(inplace=True)
df_tor.drop('index',axis=1,inplace=True)
df_tor.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [14]:
#List of  NEIGHBOUR
print(df_tor.groupby('Borough').count()['Neighbourhood'])

Borough
Central Toronto      9
Downtown Toronto    18
East Toronto         5
West Toronto         6
Name: Neighbourhood, dtype: int64


In [0]:
borough=df_tor['Borough'].unique().tolist()

In [16]:
#finding coodinates by avg latitude/longitude of the current dataset
lat=df_tor['Latitude'].mean()
lon=df_tor['Longitude'].mean()
print('Coordinates of Toronto are {},{}'.format(lat,lon))

Coordinates of Toronto are 43.66726218421052,-79.38988323421053


In [0]:
bor_col={}
for bor in borough:
    bor_col[bor]='#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

In [71]:
map_tor=folium.Map(location=[lat,lon],zoom_start=12)

for lat,lon,bor,neigh in zip(df_tor['Latitude'],df_tor['Longitude'],df_tor['Borough'],df_tor['Neighbourhood']):
    label_text=bor+" - "+neigh
    label=folium.Popup(label_text)
    folium.CircleMarker([lat,lon],radius=6,popup=label,color=bor_col[bor],fill_opacity=0.7).add_to(map_tor)
map_tor

*Getting Locations Data using FourSquare*

In [0]:
CLIENT_ID = 'F0WRGCSPH40YCKQGNCNMUJ1XN1Q0W5N4DF5XY5THICOEFT2C' # your Foursquare ID
CLIENT_SECRET = 'LZM1F3QED4GDFUP0Q5OVXV5GQ0E45DM0DTPZFGE1MOW1DRHV' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius  

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
tor_venues = getNearbyVenues(names=df_tor['Neighbourhood'],
                                latitudes=df_tor['Latitude'],
                                longitudes=df_tor['Longitude'])

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [32]:
tor_venues.shape

(1711, 7)

In [34]:
tor_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [36]:
tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,17,17,17,17,17,17


In [38]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))


There are 235 uniques categories.


In [40]:
tor_venues['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Greek Restaurant', 'Ice Cream Shop', 'Cosmetics Shop',
       'Italian Restaurant', 'Brewery', 'Yoga Studio', 'Restaurant',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Pizza Place',
       'Juice Bar', 'Bookstore', 'Indian Restaurant',
       'Furniture / Home Store', 'Grocery Store', 'Spa', 'Diner',
       'Bubble Tea Shop', 'Coffee Shop', 'Caribbean Restaurant', 'Bakery',
       'Sports Bar', 'Café', 'American Restaurant', 'Sushi Restaurant',
       'Liquor Store', 'Gym', 'Burger Joint', 'Fish & Chips Shop', 'Park',
       'Pet Store', 'Burrito Place', 'Steakhouse', 'Movie Theater',
       'Fast Food Restaurant', 'Sandwich Place', 'Food & Drink Shop',
       'Fish Market', 'Gay Bar', 'Cheese Shop', 'Chinese Restaurant',
       'Middle Eastern Restaurant', 'Thai Restaurant',
       'Comfort Food Restaurant', 'Stationery Store',
       'Seafood Restaurant', 'Coworking Space', 'Gastropub',
       'Latin American 

In [43]:
# check if the results contain "Indian Restaurant"
#please note I changed the data to Indian because I was previously writing the code using Asian but the number is so small
"Indian Restaurant" in tor_venues['Venue Category'].unique()

True

In [45]:
# one hot encoding
to_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1711, 236)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,...,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [47]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(38, 236)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,...,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.010000,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.040000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.020000,0.000000,0.000000,0.01,0.030000,...,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.020000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.030000,0.010000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.017857,0.035714,0.000000,0.000000,0.00,0.017857,0.017857,0.00,0.035714,0.000000,0.017857,0.017857,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,0.00,0.000000,...,0.000000,0.035714,0.000000,0.017857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.00,0.00,0.017857,0.000000,0.00,0.000000,0.017857,0.017857,0.000000,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.045455,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.055556,0.00,0.055556,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.07

In [49]:
len(to_grouped[to_grouped["Indian Restaurant"] > 0])

8

Dataframe to find Asian Restro only

In [54]:
to_asian = to_grouped[["Neighborhoods","Indian Restaurant"]]
to_asian.head()

,Neighborhoods,Indian Restaurant
0,"Adelaide, King, Richmond",0.01
1,Berczy Park,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.00


**CLUSTER NEIGHBORHOODS**

Run K-means to cluster the neighborhoods in Toronto in 3 Cluster


In [56]:
# set number of clusters
toclusters = 3

to_clustering = to_asian.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0], dtype=int32)

In [0]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = to_asian.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [59]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head()

,Neighborhood,Indian Restaurant,Cluster Labels
0,"Adelaide, King, Richmond",0.01,0
1,Berczy Park,0.00,0
2,"Brockton, Exhibition Place, Parkdale Village",0.00,0
3,Business Reply Mail Processing Centre 969 Eastern,0.00,0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.00,0


In [61]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(tor_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1711, 9)


,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide, King, Richmond",0.01,0,43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall
0,"Adelaide, King, Richmond",0.01,0,43.650571,-79.384568,The Keg Steakhouse & Bar,43.649937,-79.384196,Steakhouse
0,"Adelaide, King, Richmond",0.01,0,43.650571,-79.384568,Nathan Phillips Square,43.652270,-79.383516,Plaza
0,"Adelaide, King, Richmond",0.01,0,43.650571,-79.384568,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
0,"Adelaide, King, Richmond",0.01,0,43.650571,-79.384568,Shangri-La Toronto,43.649129,-79.386557,Hotel


In [62]:
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(1711, 9)


,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide, King, Richmond",0.010000,0,43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,The Beer Store,43.657773,-79.357463,Beer Store
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,TD Canada Trust,43.652780,-79.356336,Bank
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,The Abnormal Beauty Company,43.649892,-79.361005,Cosmetics Shop
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,Ontario Spring Water Sake Company,43.649922,-79.360073,Brewery
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,Vistek,43.657046,-79.359667,Electronics Store
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,Soulpepper Theatre,43.650780,-79.357615,Theater
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,Arta Gallery,43.650022,-79.361222,Art Gallery
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,Wildly Delicious Cafe,43.650436,-79.358862,Café
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,Caffe Furbo,43.649970,-79.358849,Café


In [0]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [66]:
# create map
map_clusters = folium.Map(location=[lat, lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
map_clusters.save('map_clusters.html')

**Examine Cluster**

In [68]:
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide, King, Richmond",0.010000,0,43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,The Beer Store,43.657773,-79.357463,Beer Store
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,TD Canada Trust,43.652780,-79.356336,Bank
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,The Abnormal Beauty Company,43.649892,-79.361005,Cosmetics Shop
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,Ontario Spring Water Sake Company,43.649922,-79.360073,Brewery
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,Vistek,43.657046,-79.359667,Electronics Store
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,Soulpepper Theatre,43.650780,-79.357615,Theater
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,Arta Gallery,43.650022,-79.361222,Art Gallery
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,Wildly Delicious Cafe,43.650436,-79.358862,Café
20,"Harbourfront, Regent Park",0.000000,0,43.654260,-79.360636,Caffe Furbo,43.649970,-79.358849,Café


In [69]:
to_merged.loc[to_merged['Cluster Labels'] == 1]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
37,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Bulk Barn,43.676790,-79.355865,Grocery Store
6,Central Bay Street,0.023529,1,43.657952,-79.387383,Butter Baker,43.655669,-79.384298,Bakery
37,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Urban Nails,43.676668,-79.356602,Spa
37,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Pan on the Danforth,43.678263,-79.348648,Greek Restaurant
37,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Detroit Eatery,43.677594,-79.351761,Diner
37,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Kalyvia,43.677788,-79.351127,Greek Restaurant
37,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Tsaa Tea Shop,43.677769,-79.351304,Bubble Tea Shop
37,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Marvel Coffee Co.,43.678630,-79.347460,Coffee Shop
37,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Cafe Frappe,43.678126,-79.348434,Coffee Shop
37,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Simone's Caribbean Restaurant,43.678655,-79.346582,Caribbean Restaurant


In [70]:
to_merged.loc[to_merged['Cluster Labels'] == 2]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
34,"The Annex, North Midtown, Yorkville",0.043478,2,43.67271,-79.405678,Ezra's Pound,43.675153,-79.405858,Café
34,"The Annex, North Midtown, Yorkville",0.043478,2,43.67271,-79.405678,Martino's Pizza,43.675560,-79.403558,Pizza Place
34,"The Annex, North Midtown, Yorkville",0.043478,2,43.67271,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
34,"The Annex, North Midtown, Yorkville",0.043478,2,43.67271,-79.405678,Rose & Sons,43.675668,-79.403617,American Restaurant
34,"The Annex, North Midtown, Yorkville",0.043478,2,43.67271,-79.405678,Jean Sibelius Square,43.671426,-79.408831,Park
34,"The Annex, North Midtown, Yorkville",0.043478,2,43.67271,-79.405678,Live Organic Food Bar,43.675053,-79.406715,Vegetarian / Vegan Restaurant
34,"The Annex, North Midtown, Yorkville",0.043478,2,43.67271,-79.405678,Beauty Boutique by Shoppers Drug Mart,43.674959,-79.407986,Cosmetics Shop
34,"The Annex, North Midtown, Yorkville",0.043478,2,43.67271,-79.405678,Super Jet International Coffee Shop,43.674332,-79.409325,Coffee Shop
34,"The Annex, North Midtown, Yorkville",0.043478,2,43.67271,-79.405678,Shoppers Drug Mart,43.674959,-79.407986,Pharmacy
34,"The Annex, North Midtown, Yorkville",0.043478,2,43.67271,-79.405678,Subway,43.674965,-79.406868,Sandwich Place


**OBSERVATION**

Most of the Indian Restaurants are in cluster 2 which is around The Annex, North Midtown, Yorkville and in cluster 1 around The Danforth West, Riverdale

And lowest (close to Zero) in Custer 0 areas which are around Commerce Court, Victoria Hotel and also there are good opportunities to open near Harbourfront, Regent Park, Adelaide, King, Richmond.
Looking at nearby venues it seems Cluster 0 might be a good location as there are not a lot of Asian Restaurants in these areas.

Therefore, this project recommends the entrepreneur to open a authentic INDIAN restaurant in these locations with little to no competition.
Noneteless, if the food is authentic, affordable and good taste, I am confident that it will have great following everywhere.